<a href="https://colab.research.google.com/github/rvraghvender/DeepLearningProjects/blob/main/ConvolutionNeuralNetworks/FaceRecognition/Face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Recognition

In this notebook, we are going to build face recoginition system. Many of the ideas presented in this notebook are taken from **FaceNet**.

**Face Recognition** problem falls into two categories.
- Face verification: 'Is this the claimed person?' For example, at some airports we can pass through the customs by letting system scan our passport and then verifying that the person carrying the passport is the actual person. Mobile phone that unlock using our face is also using face verficiation. This is 1:1 matching problem.

- Face Recognition: 'Who is the person?". For example, the attentance system of the employees in the enterprise using face recognition without needing to identify otherwise. This is 1:K matching problem.

FaceNet learns a neural network that encodes a face image into a vector of 128 numbers. By comparing two such vectors, we can determine if two pictures are of same person.



## 1. Packages

In [1]:
# Import relevant packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import PIL

%matplotlib inline
%load_ext autoreload
%autoreload 2

## 2. Naive Face Verification

In face verification, we have given two images and we have to determine if they are of the same person. The simplest way to do this is to compare the two images pixel-by-pixel. If the distance between the raw images is below a chosen threshold, it may represent the same person.

<img src="https://github.com/rvraghvender/DeepLearningProjects/blob/main/ConvolutionNeuralNetworks/FaceRecognition/images/pixel_comparison.png?raw=true" style="width:380px;height:150px;">
<caption><center> <u> <font color='purple'> <b>Figure 1</b> </u></center></caption>

Of course, this algorithm performs poorly, since the pixel values changes dramatically due to variation in lighthing, orientation of ther person's face, minor changes in the head position, and so on.

We will see that rather than using raw image, we can learn an enconding. By using an enconding for an image, an element-wise comparison produces a more accurate judgement as to whether two pictures are of the same person or not.

## 3. Encoding Face Images into a 128-Dimensional Vector
### 3.1 Using a ConvNet to Compute Encodings

The FaceNet model takes a lot of data and a long time to train. So following the common practice in applied deep learning, we'll load weights that someone else has already trained. The network architecture follows the Inception model from <a href='www.www.com'>Szegedy et al.</a>. Look into the module `inception_blocks_v2.py`.

The key findings to be aware of are:
- This network uses 160 X 160 dimensional RGB images as input. Specifically, a face image (or batch of $m$ face images) as a tensor of shape ($m$, $n_H$, $n_W$, $n_C$) = (m, 160, 160, 3)
- The input images are originally of shape 96 X 96, thus, we need to scale them to 160 X 160. This is done in the `img_to_encoding()` function.
-The output is a matrix of shape ($m$, 128) that encodes each input face image into 128-dimensional vector.

In [5]:
# Reading the model's pretrained weights from json file
from tensorflow.keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

In [29]:
!unzip FACE/faceNet.zip

Archive:  FACE/faceNet.zip
  inflating: facenet_keras.h5        
  inflating: model.h5                
  inflating: model.json              
  inflating: weights.h5              


In [31]:
# Loading the model
from tensorflow.keras.models import load_model
model = load_model('./facenet_keras.h5')

ValueError: ignored